## Teste de PostgrSQL



In [35]:
import psycopg2

conn = psycopg2.connect(
    dbname="detectid",
    user="postgres",
    password="Goncalo123",
    host="localhost"
)

cur = conn.cursor()
cur.execute("SELECT * FROM cdmdatabaseschema.person_details LIMIT 5;")
print(cur.fetchall())

cur.close()
conn.close()

[(1, 'Gonçalo', 'Silva'), (2, 'Violeta', 'Santos'), (3, 'Diogo', 'Lobo'), (4, 'Claudia', 'Silva'), (5, 'Diogo', 'Silva')]


In [36]:
import pandas as pd


file_path = "detectid.csv"
df = pd.read_csv(file_path)


df.head()

,Pessoa,Tempo,Dia,Hora,NIVEL DE CONSCIÊNCIA,FREQUÊNCIA CARDIACA,TA Sistólica,TA Diastólica,TEMPERATURA,SpO2,NECESSIDADE DE O2,DOR,DESCOMPENSAÇÃO,Ativação Médico,Aumento da Vigilância,Via Área Ameaçada,Evento,Area\nMonotorizada
0,Pessoa 1,0.0,11/11/2023,11:00,15,103.0,109.0,61.0,37.1,94.0,1.0,0.0,0.0,0.0,0.0,0.0,0,NaN
1,Pessoa 1,125.5,16/11/2023,18:30,15,99.0,110.0,74.0,37.1,96.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
2,Pessoa 1,127.5,16/11/2023,20:30,15,78.0,102.0,78.0,NaN,NaN,0.0,0.0,1.0,1.0,1.0,0.0,1,1.0
3,Pessoa 1,133.0,17/11/2023,02:00,15,108.0,87.0,53.0,36.8,96.0,0.0,0.0,1.0,0.0,1.0,0.0,1,1.0
4,Pessoa 1,138.5,17/11/2023,7:30,15,96.0,94.0,58.0,37.1,NaN,0.0,0.0,1.0,0.0,1.0,0.0,1,1.0


In [37]:
# Tempo tem que ser calculado logo nao entra na base de dados
# Evento foi uma experiencia, tambem nao entra
df_cleaned = df.drop(columns=["Tempo", "Evento"])

df_cleaned.rename(columns=lambda x: x.strip().replace("\n", " "), inplace=True)

# Valores Omissos preenchidos com média
numeric_cols = df_cleaned.select_dtypes(include='number').columns
df_cleaned[numeric_cols] = df_cleaned[numeric_cols].fillna(df_cleaned[numeric_cols].mean())

# Ver exemplo de dados limpos e prontos para inserção
df_cleaned.head()

,Pessoa,Dia,Hora,NIVEL DE CONSCIÊNCIA,FREQUÊNCIA CARDIACA,TA Sistólica,TA Diastólica,TEMPERATURA,SpO2,NECESSIDADE DE O2,DOR,DESCOMPENSAÇÃO,Ativação Médico,Aumento da Vigilância,Via Área Ameaçada,Area Monotorizada
0,Pessoa 1,11/11/2023,11:00,15,103.0,109.0,61.0,37.100000,94.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.088889
1,Pessoa 1,16/11/2023,18:30,15,99.0,110.0,74.0,37.100000,96.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,Pessoa 1,16/11/2023,20:30,15,78.0,102.0,78.0,36.580682,91.632432,0.0,0.0,1.0,1.0,1.0,0.0,1.000000
3,Pessoa 1,17/11/2023,02:00,15,108.0,87.0,53.0,36.800000,96.000000,0.0,0.0,1.0,0.0,1.0,0.0,1.000000
4,Pessoa 1,17/11/2023,7:30,15,96.0,94.0,58.0,37.100000,91.632432,0.0,0.0,1.0,0.0,1.0,0.0,1.000000


In [38]:
import pandas as pd
import psycopg2
from psycopg2.extras import execute_values


# Garantir que 'Dia' e 'Hora' são string
df_cleaned["Dia"] = df_cleaned["Dia"].astype(str)
df_cleaned["Hora"] = df_cleaned["Hora"].astype(str)

# Juntar as colunas 'Dia' e 'Hora' e converter para datetime com formato automático
df_cleaned["datetime"] = pd.to_datetime(df_cleaned["Dia"] + " " + df_cleaned["Hora"], dayfirst=True, errors="coerce")

df_cleaned["person_id"] = df_cleaned["Pessoa"].str.extract(r"(\d+)").astype(int)

# Conectar à base de dados
conn = psycopg2.connect(
    dbname="detectid",
    user="postgres",
    password="Goncalo123",
    host="localhost",
    port="5432"
)

cur = conn.cursor()

# Exemplo de inserção na tabela MEASUREMENT
measurement_concepts = {
    "TA Sistólica": 1,
    "TA Diastólica": 2,
    "TEMPERATURA": 3,
    "SpO2": 4,
    "DOR": 5,
    "NECESSIDADE DE O2": 6,
    "FREQUÊNCIA CARDIACA": 7,
    "NIVEL DE CONSCIÊNCIA": 8
}


measurement_id = 1
rows = []
for _, row in df_cleaned.iterrows():
    for field, concept_id in measurement_concepts.items():
        rows.append((
            measurement_id,
            int(row["person_id"]),
            concept_id,
            row[field],
            row["datetime"]
        ))
        measurement_id += 1

query = """
INSERT INTO cdmDatabaseSchema.MEASUREMENT (
    measurement_id, person_id, measurement_concept_id, value_as_number, measurement_datetime
) VALUES %s
"""


execute_values(cur, query, rows)
conn.commit()

print("Dados inseridos com sucesso!")
cur.close()
conn.close()


InvalidTextRepresentation: invalid input syntax for type numeric: " "
LINE 4: ...,7,66.0,'2023-10-27T07:00:00'::timestamp),(56,2,8,' ','2023-...
                                                             ^
